In [ ]:
#|hide
#| eval: false
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
#|default_exp callback.galileo

In [ ]:
#|export
import os

from __future__ import annotations

import pandas as pd
import numpy as np

from functools import partial

from torch.nn import Module
from typing import Any, Callable, Dict, List

from fastai.learner import Callback
from fastai.data.load import DataLoader

In [ ]:
#|hide
from nbdev.showdoc import *

# Galileo

> Integration with [Galileo](https://docs.rungalileo.io/integrations/fastai) 

To use Galileo on your dataset need to install the dataquality client and register your account.
```
pip install dataquality
```


1. Create account: [rungalileo.io](https://console.cloud.rungalileo.io/sign-up).
2. Grab your [token](https://console.cloud.rungalileo.io/get-token). 



In [ ]:
#|export
import dataquality

In [ ]:
#|export

class _PatchDLGetIdxs:
    """
    Patch the DataLoader to store the indices of the batches.
    For example:
    self.dl.get_idxs = _PatchDLGetIdxs(self.dl.get_idxs, self.idx_log)
    """

    def __init__(self, old_func: Callable, store: Dict[str, Any]):
        """
        Patch the DataLoader to store the indices of the batches.
        For example:
        self.dl.get_idxs = IdxLogPatch(self.dl.get_idxs, self.idx_log)
        :param old_func: The original function to patch.
        :param store: The store to store the indices in.
        """
        self.old_func = old_func
        self.store = store
        self.store["dataloader_indices"] = []

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        """
        Call the original function and store the indices.
        :param args: The arguments to pass to the original function.
        :param kwargs: The keyword arguments to pass to the original function.

        """
        res = self.old_func(*args, **kwargs)
        if res:
            self.store["dataloader_indices"].append(res)
        return res

In [ ]:
# |export
class DataqualityCallback(Callback):
    """
    Dataquality logs the model embeddings and logtis to measure the quality of the dataset.
    Provide the label names and the classifier layer to log the embeddings and logits.
    If no classifier layer is provided, the last layer of the model will be used.
    Here is how to take the last layer of the model:
    dqc = DataqualityCallback(labels=['negative','positive'], layer=model.fc)
    """

    hook = None
    is_initialized = False
    labels = None

    def __init__(
        self,
        labels: List[str],
        layer: Any = None,
        log_dataset: bool = True,
        task_type: str = "image_classification",
        options: Dict[str, Any] = {},
        *args: Any,
        **kwargs: Any,
    ) -> None:
        """
        Dataquality logs the model embeddings and logits to measure the quality
        of the dataset. This helps to find mislabelled samples in a data centric approach.
        :param layer: Classifier layer with embeddings as input and logits as output.
        :param log_dataset: Enable automatic extraction of the dataset to data quality.
        :param disable_dq: Disable data quality logging.
        :param args: The arguments to pass to the super class.
        :param kwargs: The keyword arguments to pass to the super class.
        """
        super().__init__(*args, **kwargs)
        self.disable_dq = os.environ.get("DQ_NOOP", False)
        self.labels = labels
        self.log_dataset = log_dataset
        self.options = options
        self.layer = layer
        self.model_outputs_log = {}
        self.current_idx = []
        self.idx_store = {"idx_queue": []}
        self.logger_config = {}
        self.counter = 0
        self.options["task_type"] = task_type

        if self.labels is None:
            raise ValueError(
                """Labels must be provided. For example: 
          DataqualityCallback(labels=['negative','positive'])"""
            )

        if not self.disable_dq:
            dataquality.init(**options)
            self.logger_config = dataquality.get_model_logger().logger_config

    def get_layer(self) -> Module:
        """
        Get the classifier layer, which inputs and outputs will be logged
        (embeddings and logits).
        :return: The classifier layer.
        """
        if self.layer is None:
            # Get the last layer of the model
            return list(list(self.model.children())[-1].children())[-1]
        else:
            return self.layer

    def before_epoch(self) -> None:
        if not self.disable_dq:
            dataquality.set_epoch(self.epoch)

    def before_fit(self) -> None:
        if self.is_initialized or self.disable_dq:
            return
        self.wrap_indices()
        self.register_hooks()
        self.log_data()
        self.is_initialized = True

    def before_train(self) -> None:
        """
        Sets the split in data quality and registers the classifier layer hook.
        """
        if self.disable_dq:
            return
        dataquality.set_split(dataquality.schemas.split.Split.train)
        self.wrap_indices()
        if self.is_initialized:
            return
        self.register_hooks()
        self.log_data()
        self.is_initialized = True

    def log_data(self):
        """
        Log datasets to dataquality
        """
        if self.disable_dq or not self.log_dataset:
            return

        dataquality.set_labels_for_run(self.labels)
        num_datasets = self.dls.n_subsets
        train_dl, valid_dl, test_dl = None, None, None
        if num_datasets == 1:
            train_dl = self.dls
        elif num_datasets == 2:
            train_dl, valid_dl = self.dls
        elif num_datasets == 3:
            train_dl, valid_dl, test_dl = self.dls
        if self.options.get("task_type") == "image_classification":
            if train_dl is not None:
                dataquality.log_image_dataset(
                    self.convert_img_dl_to_df(train_dl),
                    imgs_colname="image",
                    imgs_location_colname="path",
                    split=dataquality.schemas.split.Split.training,
                )
            if valid_dl is not None:
                dataquality.log_image_dataset(
                    self.convert_img_dl_to_df(valid_dl),
                    imgs_colname="image",
                    imgs_location_colname="path",
                    split=dataquality.schemas.split.Split.validation,
                )
        else:
            if train_dl is not None:
                dataquality.log_dataset(
                    self.convert_tab_dl_to_df(train_dl),
                    split=dataquality.schemas.split.Split.training,
                )
            if valid_dl is not None:
                dataquality.log_dataset(
                    self.convert_tab_dl_to_df(valid_dl),
                    split=dataquality.schemas.split.Split.validation,
                )

    def wrap_indices(self) -> None:
        """
        Wraps the get_idxs function of the dataloader to store the indices.
        """
        if not hasattr(self, "dl"):
            return
        if not isinstance(self.dl.get_idxs, _PatchDLGetIdxs):
            self.dl.get_idxs = _PatchDLGetIdxs(self.dl.get_idxs, self.idx_store)

    def after_validate(self):
        dataquality.set_split(dataquality.schemas.split.Split.train)

    def before_validate(self):
        """
        Sets the split in data quality and registers the classifier layer hook.
        """
        self.wrap_indices()
        if self.disable_dq:
            return
        self.disable_dq = True
        dataquality.set_split(dataquality.schemas.split.Split.validation)
        self.idx_store["idx_queue"] = []

    def after_fit(self):
        """
        Uploads data to galileo and removes the classifier layer hook.
        """
        if (self.n_epoch - 1) == self.epoch:
            self.counter += 1

        if self.counter != 2:
            return
        print("Finishing dataquality")
        try:
            self.h.remove()
        except Exception:
            pass
        dataquality.finish()

    def before_batch(self):
        """
        Clears the model outputs log.
        """
        self.model_outputs_log.clear()

    # def after_batch(self) -> None:
    def after_pred(self) -> None:
        """
        Logs the model outputs.
        """
        # Get the current batch size
        bs_len = len(self.model_outputs_log["model_output"])
        # Store the current batch ids by trimming the stored ids by
        # the batch size length
        indices = self.idx_store["dataloader_indices"][-1][:bs_len].copy()
        idx_store = self.idx_store["dataloader_indices"][-1][bs_len:]
        self.idx_store["dataloader_indices"][-1] = idx_store
        try:
            cur_split = self.logger_config.cur_split
            if cur_split:
                cur_split.lower()
            else:
                print("Callback could not detect split")
            id_map = self.logger_config.idx_to_id_map[cur_split]
            ids = np.array([id_map[i] for i in indices])
        except Exception as e:
            print("cur_split error", cur_split, e)
            return
        # Log the model outputs
        embs = self.model_outputs_log["model_input"][0].detach().cpu().numpy()
        logits = self.model_outputs_log["model_output"].detach().cpu().numpy()
        equal_len = len(embs) == len(logits) and len(ids) == len(embs)
        if not equal_len:
            print(
                f"length not equal. logits: {len(logits)},ids: {len(ids)},\
embs: {len(embs)}"
            )
        if self.disable_dq or not equal_len:
            return
        dataquality.log_model_outputs(embs=embs, logits=logits, ids=ids)

    def register_hooks(self) -> None:
        """
        Registers the classifier layer hook.
        """
        h = None
        if not self.hook:
            forward_hook = partial(self.forward_hook_with_store, self.model_outputs_log)
            h = self.get_layer().register_forward_hook(forward_hook)
            self.hook = h
        return h

    def forward_hook_with_store(
        self, store: Dict[str, Any], layer: Module, model_input: Any, model_output: Any
    ) -> None:
        """
        Forward hook to store the output of a layer.
        :param store: Dictionary to store the output in.
        :param layer: Layer to store the output of.
        :param model_input: Input to the model.
        :param model_output: Output of the model.
        :return: None
        """
        store["model_input"] = model_input
        store["model_output"] = model_output

    def convert_img_dl_to_df(
        self, dl: DataLoader, x_col: str = "image"
    ) -> pd.DataFrame:
        """
        Converts a fastai DataLoader to a pandas DataFrame.
        :param dl: Fast ai DataLoader to convert.
        :param x_col: Name of the column to use for the x values, for example image.
        :return: Pandas DataFrame with the data from the DataLoader.
        """
        additional_data = {}
        if x_col == "image":
            additional_data["path"] = dl.items
        x, y = [], []
        for x_item, y_item in dl.dataset:
            x.append(x_item)
            y.append(int(y_item))
        ids = dl.vocab.o2i.keys()
        if len(ids) == 2 and isinstance(next(iter(ids)), bool):
            ids = dl.dataset.splits[dl.dataset.split_idx]
        df = pd.DataFrame({"id": ids, x_col: x, "label": y, **additional_data})
        del additional_data, x, y
        return df

    def convert_tab_dl_to_df(
        self, dl: DataLoader, x_col: str = "text", y_col="label"
    ) -> pd.DataFrame:
        """
        Converts a fastai DataLoader to a pandas DataFrame.
        :param dl: Fast ai DataLoader to convert.
        :param x_col: Name of the column to use for the x values, for example text.
        :param y_col: Name of the column to use for the y values, for example label.
        :return: Pandas DataFrame with the data from the DataLoader.
        """
        df = dl.items.copy()
        rename_options = {}
        if "x_col" != "text":
            rename_options[x_col] = "text"
        if "y_col" != "text":
            rename_options[x_col] = "label"
        if rename_options:
            df = df.rename(columns=rename_options)
        if "id" not in df.columns:
            df["id"] = df.index
        return df


In [ ]:
show_doc(DataqualityCallback)

---

### DataqualityCallback

>      DataqualityCallback (labels:List[str], layer:Any=None,
>                           log_dataset:bool=True,
>                           task_type:str='image_classification',
>                           options:Dict[str,Any]={}, *args:Any, **kwargs:Any)

Dataquality logs the model embeddings and logtis to measure the quality of the dataset.
Provide the label names and the classifier layer to log the embeddings and logits.
If no classifier layer is provided, the last layer of the model will be used.
Here is how to take the last layer of the model:
dqc = DataqualityCallback(labels=['negative','positive'], layer=model.fc)

In [ ]:
#|export
_all_ = []

## Export -

In [ ]:
#|hide
from nbdev import *
nbdev_export()